In [ ]:
import model
import utils
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data
from torch import optim
from torch.optim import lr_scheduler
from torch.nn import init
from pathlib import Path
import torch.nn.functional as F
import itertools
import random
import glob
import os
import torchvision.utils as vutils


# argparser
parser = argparse.ArgumentParser()
parser.add_argument("--dataroot", type=str, default="../datasets", help="path to folder containing datasets")
parser.add_argument("--dataset", type=str, default="human2anime", help="name of dataset (default: 'human2anime')")
parser.add_argument("--n-epochs", type=int, default=200, help="total number of training epochs")
parser.add_argument("--decay-epoch", type=int, default=100, help="epoch to start linearly decaying learning rate")
parser.add_argument("--image-size", type=int, default=256, help='pixel-size of the image')
parser.add_argument("-b", "--batch-size", type=int, default=1, help="batch size")
parser.add_argument("--lr", type=float, default=0.0002, help="learning rate")
parser.add_argument("--out", type=str, default="./output", help="output path")
parser.add_argument("--cuda", type=utils.str2bool, nargs='?', const=True, default=True,
                    help='true/false value indicating whether to use cuda')
parser.add_argument("--input_nc", type=int, default=3, help='number of channels of input data')
parser.add_argument("--output_nc", type=int, default=3, help='number of channels of output data')
parser.add_argument("--log-freq", type=int, default=200, help='frequency of printing losses to stdout')
parser.add_argument("--visdom-freq", type=int, default=1000, help='frequency of showing training results in visdom')
parser.add_argument("--save-freq", type=int, default=1000, help="frequency to save images")
parser.add_argument("--visdom", type=utils.str2bool, nargs='?', const=True, default=False,
                    help='true/false value indicating whether to use visdom')
parser.add_argument("--save-images", type=utils.str2bool, nargs='?', const=True, default=True,
                    help='true/false value indicating whether to save images generated during training')
parser.add_argument("--verbose", type=utils.str2bool, nargs='?', const=True, default=True,
                    help='true/false value indicating whether to use tqdm')
parser.add_argument("--log", type=utils.str2bool, nargs='?', const=True, default=False,
                    help='true/false value indicating whether to log losses during training if not using tqdm')
parser.add_argument("--name", type=str, help='the unique directory name for each experiment')
parser.add_argument("--weights_epoch", type=int, default=199, help='epoch of the weights to load')
parser.add_argument("--manualSeed", type=int, help="seed for training")
args = parser.parse_args()

# unique dir for outputs, weights, and results
if args.name is None:
    unique_dir = f'{args.n_epochs}{args.batch_size}{args.lr}{args.image_size}'
else:
    unique_dir = args.name

print(f'Experiment name: {unique_dir}')

# create directories for results
try:
    os.makedirs(os.path.join("./results", args.dataset, unique_dir, "A"))
    os.makedirs(os.path.join("./results", args.dataset, unique_dir, "B"))
except OSError:
    pass

# directory for weights
weights_dir = os.path.join("./weights", args.dataset, unique_dir)

# define transformations
data_transform = transforms.Compose([
                    transforms.Resize(int(args.image_size * 1.12), Image.BICUBIC),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

# create datasets
dataset_A = datasets.ImageFolder(root=os.path.join(args.dataroot, args.dataset, "test", "A"), 
                                 transform=data_transform)
dataset_B = datasets.ImageFolder(root=os.path.join(args.dataroot, args.dataset, "test", "B"), 
                                 transform=data_transform)

# create dataloader (note: pin_memory=True makes transferring samples to GPU faster)
dataloader_A = DataLoader(dataset_A, batch_size=args.batch_size, pin_memory=args.cuda)
dataloader_B = DataLoader(dataset_B, batch_size=args.batch_size, pin_memory=args.cuda)

# set device to cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# create models
g_AB = model.CycleGAN().to(device)
g_BA = model.CycleGAN().to(device)

# Load state dicts
g_AB.load_state_dict(torch.load(os.path.join(weights_dir, f'g_AB_epoch_{args.weights_epoch}.pth')))
g_BA.load_state_dict(torch.load(os.path.join(weights_dir, f'g_BA_epoch_{args.weights_epoch}.pth')))

# Set models to eval mode
g_AB.eval()
g_BA.eval()

# human to anime loop
for i, (data, target) in enumerate(dataloader_A):
    # get images
    real_img_A = data.to(device)

    # Generate output
    fake_img_B = 0.5 * (g_AB(real_img_A).data + 1.0)

    # Save image files
    vutils.save_image(real_img_A.detach(), f"./results/{args.dataset}/{unique_dir}/A/real_{i}.png", normalize=True)
    vutils.save_image(fake_img_B.detach(), f"./results/{args.dataset}/{unique_dir}/A/fake_{i}.png", normalize=True)
print(f'{len(dataloader_A)} human to anime images generated')    

# anime to human loop
for i, (data, target) in enumerate(dataloader_B):
    # get images
    real_img_B = data.to(device)

    # Generate output
    fake_img_A = 0.5 * (g_BA(real_img_B).data + 1.0)

    # Save image files
    vutils.save_image(real_img_B.detach(), f"./results/{args.dataset}/{unique_dir}/B/real_{i}.png", normalize=True)
    vutils.save_image(fake_img_A.detach(), f"./results/{args.dataset}/{unique_dir}/B/fake_{i}.png", normalize=True)
print(f'{len(dataloader_B)} anime to human images generated')  

In [3]:
import os

mydir = os.path.join("./results", "A")
str(mydir)

'./results\\A'

In [2]:
os.path.join(mydir, 'extra_dir')

'./results\\A\\extra_dir'